## code for colab and install library

In [ ]:
#######for colab##########
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

id_dic = {
    'train.csv':'', 
    'test.csv':'', 
    'stores.csv':'', 
    'sample_submission.csv':'',
    'goods.csv':'', 
    'genres.csv':'', 
    }

for k,v in id_dic.items():
    downloaded = drive.CreateFile({'id': v})
    downloaded.GetContentFile(f'{k}')

##############################################
!pip install category_encoders
!pip install shap
!pip install optuna
!pip install catboost
!pip3 uninstall --yes lightgbm && pip3 install --install-option=--gpu lightgbm

## import library and function

In [ ]:
import pandas as pd
import numpy as np
from itertools import product

import altair as alt 
import os

from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import category_encoders as ce

pd.set_option('display.max_rows', 30)
pd.set_option('display.max_columns', 130)
pd.set_option('display.max_colwidth', 50)

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn

import datetime

import lightgbm as lgb 
from sklearn import datasets
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error 
from sklearn.metrics import r2_score 

from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import nltk
import re

from sklearn import preprocessing
from itertools import product
from tqdm import tqdm_notebook
import xgboost as xgb
from matplotlib import rcParams
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, train_test_split
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor, Pool

# =====function============
def kesson_table(df):
    null_val = df.isnull().sum()
    percent = 100 * null_val/len(df)
    kesson_table = pd.concat([null_val, percent], axis = 1)
    kesson_table_ren_columns = kesson_table.rename(columns = {0: "欠損数", 1: "%"})
    return kesson_table_ren_columns

#https://qiita.com/mizutaku17/items/6bd8e778b6e9ec7b162d
def outlier_2s(col):
    average = np.mean(col)
    sd = np.std(col)
    me = np.median(col)
    # me = np.mean(col)
    # me = np.mode(col)
    outlier_min = average - (sd) * 2
    outlier_max = average + (sd) * 2

    col[col < outlier_min] = me
    col[col > outlier_max] = me
    return col

def outlier_iqr(col):
    q1 = col.describe()['25%']
    q3 = col.describe()['75%']
    iqr = q3 - q1 
    me = np.median(col)
    outlier_min = q1 - (iqr) * 1.5
    outlier_max = q3 + (iqr) * 1.5
    col[col < outlier_min] = me 
    col[col > outlier_max] = me 
    return col 

def price_outlier_2s(col):
    average = np.mean(col)
    sd = np.std(col)

    outlier_min = average - (sd) * 2
    outlier_max = average + (sd) * 2

    col[col < outlier_min] = None 
    col[col > outlier_max] = None 
    return col

def price_outlier_iqr(col):
    q1 = col.describe()['25%']
    q3 = col.describe()['75%']
    iqr = q3 - q1 
    me = np.median(col)
    outlier_min = q1 - (iqr) * 1.5
    outlier_max = q3 + (iqr) * 1.5
    col[col < outlier_min] = None 
    col[col > outlier_max] = None
    return col 

def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols =   [c for c in df if df[c].dtype == "int64"]
    
    df[float_cols] = df[float_cols].astype(np.float16)
    df[int_cols]   = df[int_cols].astype(np.int16)
    return df

def generate_lag(train, months, lag_column):
    for month in months:
        train_shift = train[['num_month', 'store_id', 'goods_id', lag_column]].copy()
        train_shift.columns = ['num_month', 'store_id', 'goods_id', lag_column+'_lag_'+ str(month)]
        train_shift['num_month'] += month
        train = pd.merge(train, train_shift, on=['num_month', 'store_id', 'goods_id'], how='left')
    return train

## load data

In [ ]:
train = pd.read_csv('train.csv')
train['yy_mm_dd'] = train['yy_mm_dd'].apply(lambda x: str(x).replace('25-','2013-').replace('26-','2014-').replace('27-','2015-'))
train['yy_mm_dd'] = train['yy_mm_dd'].apply(lambda x: f'{x[8:10]}.{x[5:7]}.{x[:4]}')
train['num_month'] = train['num_month'].apply(lambda x: 34+x-1)

test = pd.read_csv('test.csv')

store = pd.read_csv('stores.csv')

submission = pd.read_csv('sample_submission.csv')

genres = pd.read_csv('genres.csv')

goods = pd.read_csv('goods.csv')

print(f'train:{train.shape}')
print(f'test:{test.shape}')
print(f'store:{store.shape}')
print(f'submission:{submission.shape}')
print(f'genres:{genres.shape}')
print(f'goods:{goods.shape}')

train:(2882335, 6)
test:(214200, 3)
store:(60, 2)
submission:(214200, 2)
genres:(84, 2)
goods:(22170, 3)


## preprocessing

In [ ]:
train = train[train['price'] < 30000000]
price_correction = train[(train['store_id'] == 33) & (train['goods_id'] == 2974) & (train['num_month'] == 4) & (train['price'] > 0)].price.median()
train.loc[train['price'] < 0, 'price'] = price_correction

train.loc[train['store_id'] == 1, 'store_id'] = 58
test.loc[test['store_id'] == 1, 'store_id'] = 58
train.loc[train['store_id'] == 2, 'store_id'] = 59
test.loc[test['store_id'] == 2, 'store_id'] = 59
train.loc[train['store_id'] == 11, 'store_id'] = 12
test.loc[test['store_id'] == 11, 'store_id'] = 12

store.loc[0,'store_name'] = ' '.join(store.loc[0,'store_name'].split(' ')[1:])
store.loc[1,'store_name'] = ' '.join(store.loc[1,'store_name'].split(' ')[1:])
store['city'] = store['store_name'].str.split(' ').map(lambda row: row[0])
le = preprocessing.LabelEncoder()
le.fit_transform(store['city'])
store['city_label'] = le.fit_transform(store['city'])
store.drop(['store_name', 'city'], axis = 1, inplace = True)

le = preprocessing.LabelEncoder()
main_categories = genres['goods_genre_name'].str.split('-')
genres['main_category_id'] = main_categories.map(lambda row: row[0].strip())
genres['main_category_id'] = le.fit_transform(genres['main_category_id'])

genres['sub_category_id'] = main_categories.map(lambda row: row[1].strip() if len(row) > 1 else row[0].strip())
genres['sub_category_id'] = le.fit_transform(genres['sub_category_id'])

train['yy_mm_dd'] =  pd.to_datetime(train['yy_mm_dd'], format='%d.%m.%Y')

months = train['num_month'].unique()
cartesian = []
for month in months:
    shops_in_month = train.loc[train['num_month']==month, 'store_id'].unique()
    items_in_month = train.loc[train['num_month']==month, 'goods_id'].unique()
    cartesian.append(np.array(list(product(*[shops_in_month, items_in_month, [month]])), dtype='int32'))
cartesian_df = pd.DataFrame(np.vstack(cartesian), columns = ['store_id', 'goods_id', 'num_month'], dtype=np.int32)
x = train.groupby(['store_id', 'goods_id', 'num_month'])['units_sold_day'].sum().rename('units_sold_month').reset_index()
new_train = pd.merge(cartesian_df, x, on=['store_id', 'goods_id', 'num_month'], how='left').fillna(0)

In [ ]:
del x
del cartesian_df
del cartesian
del train

In [ ]:
new_train.sort_values(['num_month','store_id','goods_id'], inplace = True)
test.insert(loc=3, column='num_month', value=33)
test['units_sold_month'] = 0
new_train = new_train.append(test.drop('index', axis = 1))
new_train = pd.merge(new_train, store, on=['store_id'], how='left')
new_train = pd.merge(new_train, goods.drop('goods_name', axis = 1), on=['goods_id'], how='left')
new_train = pd.merge(new_train, genres.drop('goods_genre_name', axis = 1), on=['goods_genre_id'], how='left')

In [ ]:
del goods
del genres
del store 
del test

In [ ]:
new_train = downcast_dtypes(new_train)

In [ ]:
import gc
gc.collect()

11

In [ ]:
%%time
new_train = generate_lag(new_train, [1,2,3,4,5,6,12], 'units_sold_month')

CPU times: user 1min 2s, sys: 709 ms, total: 1min 3s
Wall time: 1min 3s


In [ ]:
%%time
group = new_train.groupby(['num_month', 'goods_id'])['units_sold_month'].mean().rename('item_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['num_month', 'goods_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'item_month_mean')
new_train.drop(['item_month_mean'], axis=1, inplace=True)

CPU times: user 58.7 s, sys: 806 ms, total: 59.5 s
Wall time: 59.5 s


In [ ]:
%%time
group = new_train.groupby(['num_month', 'store_id'])['units_sold_month'].mean().rename('shop_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['num_month', 'store_id'], how='left')
new_train = generate_lag(new_train, [1,2,3,6,12], 'shop_month_mean')
new_train.drop(['shop_month_mean'], axis=1, inplace=True)

CPU times: user 1min 4s, sys: 667 ms, total: 1min 5s
Wall time: 1min 5s


In [ ]:
%%time
group = new_train.groupby(['num_month', 'store_id', 'goods_genre_id'])['units_sold_month'].mean().rename('shop_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['num_month', 'store_id', 'goods_genre_id'], how='left')
new_train = generate_lag(new_train, [1, 2], 'shop_category_month_mean')
new_train.drop(['shop_category_month_mean'], axis=1, inplace=True)

CPU times: user 32.7 s, sys: 364 ms, total: 33 s
Wall time: 33 s


In [ ]:
%%time
group = new_train.groupby(['num_month', 'main_category_id'])['units_sold_month'].mean().rename('main_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['num_month', 'main_category_id'], how='left')
new_train = generate_lag(new_train, [1], 'main_category_month_mean')
new_train.drop(['main_category_month_mean'], axis=1, inplace=True)

CPU times: user 21 s, sys: 273 ms, total: 21.3 s
Wall time: 21.3 s


In [ ]:
%%time
group = new_train.groupby(['num_month', 'sub_category_id'])['units_sold_month'].mean().rename('sub_category_month_mean').reset_index()
new_train = pd.merge(new_train, group, on=['num_month', 'sub_category_id'], how='left')
new_train = generate_lag(new_train, [1], 'sub_category_month_mean')
new_train.drop(['sub_category_month_mean'], axis=1, inplace=True)

CPU times: user 21.3 s, sys: 252 ms, total: 21.5 s
Wall time: 21.5 s


In [ ]:
new_train['month'] = new_train['num_month'] % 12

In [ ]:
holiday_dict = {0: 6, 1: 3, 2: 2, 3: 8, 4: 3, 5: 3, 6: 2, 7: 8, 8: 4, 9: 8, 10: 5, 11: 4,}
new_train['holidays_in_month'] = new_train['month'].map(holiday_dict)

In [ ]:
new_train.to_pickle('new_train.pkl')

## model training(xgb+lgb+cat)

In [ ]:
new_train = pd.read_pickle('new_train.pkl')

In [ ]:
new_train = new_train[new_train.num_month > 11]

def fill_na(df):
    for col in df.columns:
        if ('_lag_' in col) & (df[col].isnull().any()):
            df[col].fillna(0, inplace=True)         
    return df

new_train = fill_na(new_train)

X_train = new_train[new_train.num_month < 32].drop(['units_sold_month'], axis=1).values
y_train = new_train[new_train.num_month < 32]['units_sold_month'].values

X_valid = new_train[new_train.num_month == 32].drop(['units_sold_month'], axis=1).values
y_valid = new_train[new_train.num_month == 32]['units_sold_month'].values

X_test = new_train[new_train.num_month == 33].drop(['units_sold_month'], axis=1).values

# del new_train

In [ ]:
new_train.isnull().sum().sum()

0

In [ ]:
new_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9423985 entries, 6812767 to 16236751
Data columns (total 31 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   store_id                        int16  
 1   goods_id                        int16  
 2   num_month                       int16  
 3   units_sold_month                float16
 4   city_label                      int16  
 5   goods_genre_id                  int16  
 6   main_category_id                int16  
 7   sub_category_id                 int16  
 8   units_sold_month_lag_1          float16
 9   units_sold_month_lag_2          float16
 10  units_sold_month_lag_3          float16
 11  units_sold_month_lag_4          float16
 12  units_sold_month_lag_5          float16
 13  units_sold_month_lag_6          float16
 14  units_sold_month_lag_12         float16
 15  item_month_mean_lag_1           float16
 16  item_month_mean_lag_2           float16
 17  item_month_mean_lag_

In [ ]:
import gc
gc.collect()

0

In [ ]:
def xgbtrain():

    param={
        'n_estimators':5000,
        'learning_rate':0.01,
        'max_depth':10,
        'subsample':0.5,
        'colsample_bytree':0.5,
        'tree_method':'gpu_hist',
        'gpu_id':0,
        'seed':42,
    }

    regressor = xgb.XGBRegressor(**param)
    
    regressor_ = regressor.fit(
                               X_train,
                               y_train, 
                               eval_metric = 'rmse', 
                               eval_set = [(X_train, y_train), (X_valid, y_valid)], 
                               verbose=True,
                               early_stopping_rounds = 50,
                              )
    return regressor_

In [ ]:
%%time
xgb_regressor_ = xgbtrain()
xgb_pred_v1 = xgb_regressor_.predict(X_test)

[14:15:20] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:2.44536	validation_1-rmse:4.96552
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 50 rounds.
[1]	validation_0-rmse:2.43291	validation_1-rmse:4.96037
[2]	validation_0-rmse:2.42017	validation_1-rmse:4.95447
[3]	validation_0-rmse:2.40839	validation_1-rmse:4.95179
[4]	validation_0-rmse:2.39628	validation_1-rmse:4.94952
[5]	validation_0-rmse:2.38506	validation_1-rmse:4.94586
[6]	validation_0-rmse:2.37316	validation_1-rmse:4.94304
[7]	validation_0-rmse:2.36183	validation_1-rmse:4.93701
[8]	validation_0-rmse:2.3509	validation_1-rmse:4.93556
[9]	validation_0-rmse:2.33866	validation_1-rmse:4.93137
[10]	validation_0-rmse:2.32657	validation_1-rmse:4.92722
[11]	validation_0-rmse:2.316	validation_1-rmse:4.92453
[12]	validation_0-rmse:2.30485	validation_1

In [ ]:
def lgbtrain():
    lgb_train = lgb.Dataset(X_train, y_train)
    lgb_eval = lgb.Dataset(X_valid, y_valid)

    param = {
            'n_estimators' : 5000,
            'task' : 'train',
            'boosting_type' : 'gbdt',
            'learning_rate' : 0.01,
            'bagging_freq': 5,
            'device': 'gpu',
            'seed':42,
    }
    regressor = LGBMRegressor(**param)
    regressor_ = regressor.fit(
                               X_train,
                               y_train, 
                               eval_metric = 'rmse', 
                               eval_set = [(X_train, y_train), (X_valid, y_valid)], 
                               verbose=True,
                               early_stopping_rounds = 50,
                               )
    return regressor_ 

In [ ]:
%%time
lgb_regressor_ = lgbtrain()

[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[1]	training's rmse: 2.42883	training's l2: 5.89924	valid_1's rmse: 4.95703	valid_1's l2: 24.5721
Training until validation scores don't improve for 50 rounds
[2]	training's rmse: 2.41643	training's l2: 5.83915	valid_1's rmse: 4.95341	valid_1's l2: 24.5363
[3]	training's rmse: 2.4042	training's l2: 5.78019	valid_1's rmse: 4.94903	valid_1's l2: 24.4929
[4]	training's rmse: 2.39216	training's l2: 5.72242	valid_1's rmse: 4.94447	valid_1's l2: 24.4478
[5]	training's rmse: 2.38026	training's l2: 5.66563	valid_1's rmse: 4.94104	valid_1's l2: 24.4139
[6]	training's rmse: 2.36859	training's l2: 5.61023	valid_1's rmse: 4.93661	valid_1's l2: 24.3702
[7]	training's rmse: 2.35704	training's l2: 5.55564	valid_1's rmse: 4.93333	valid_1's l2: 24.3378
[8]	training's rmse: 2.34572	training's l2: 5.50239	valid_1's rmse: 4.92897	valid_1's l2: 24.2947
[9]	training's rmse: 2.33452	training's l2: 5.44

In [ ]:
lgb_pred_v1 = lgb_regressor_.predict(X_test)

In [ ]:
def cattrain():
    param = {
           'learning_rate':0.01,
           'num_boost_round': 5000,
           'task_type':"GPU",
           'devices':'0:1',
           'random_seed':42,
    }
    regressor = CatBoostRegressor(**param)
    regressor_ = regressor.fit(
            Pool(X_train.astype(np.float32),y_train.astype(np.float32)),
            eval_set=Pool(X_valid.astype(np.float32),y_valid.astype(np.float32)),
            early_stopping_rounds=50,
            verbose=True,
    )
    return regressor_

In [ ]:
%%time
cat_regressor_ = cattrain()

0:	learn: 2.4310974	test: 4.9577157	best: 4.9577157 (0)	total: 27.9ms	remaining: 2m 19s
1:	learn: 2.4240447	test: 4.9559350	best: 4.9559350 (1)	total: 57ms	remaining: 2m 22s
2:	learn: 2.4169066	test: 4.9541298	best: 4.9541298 (2)	total: 84ms	remaining: 2m 19s
3:	learn: 2.4097230	test: 4.9513412	best: 4.9513412 (3)	total: 112ms	remaining: 2m 19s
4:	learn: 2.4000839	test: 4.9476014	best: 4.9476014 (4)	total: 140ms	remaining: 2m 19s
5:	learn: 2.3935992	test: 4.9455883	best: 4.9455883 (5)	total: 168ms	remaining: 2m 20s
6:	learn: 2.3845448	test: 4.9419379	best: 4.9419379 (6)	total: 199ms	remaining: 2m 22s
7:	learn: 2.3779515	test: 4.9402342	best: 4.9402342 (7)	total: 227ms	remaining: 2m 21s
8:	learn: 2.3693170	test: 4.9379466	best: 4.9379466 (8)	total: 254ms	remaining: 2m 20s
9:	learn: 2.3628804	test: 4.9362760	best: 4.9362760 (9)	total: 282ms	remaining: 2m 20s
10:	learn: 2.3545255	test: 4.9338281	best: 4.9338281 (10)	total: 308ms	remaining: 2m 19s
11:	learn: 2.3467015	test: 4.9291693	best:

In [ ]:
cat_pred_v1 = cat_regressor_.predict(X_test.astype(np.float32)) 

In [ ]:
en_df = pd.read_csv('sample_submission.csv')
en_df['xgb_pred_v1'] = xgb_pred_v1
en_df['lgb_pred_v1'] = lgb_pred_v1.astype(np.float32)
en_df['cat_pred_v1'] = cat_pred_v1.astype(np.float32)
en_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214200 entries, 0 to 214199
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   index             214200 non-null  int64  
 1   units_sold_month  214200 non-null  float64
 2   xgb_pred_v1       214200 non-null  float32
 3   lgb_pred_v1       214200 non-null  float32
 4   cat_pred_v1       214200 non-null  float32
dtypes: float32(3), float64(1), int64(1)
memory usage: 5.7 MB


In [ ]:
en_df.head()

,index,units_sold_month,xgb_pred_v1,lgb_pred_v1,cat_pred_v1
0,0,0.5,0.155166,0.175729,0.130323
1,1,0.5,0.009131,0.040082,0.116019
2,2,0.5,0.298522,0.308958,0.219616
3,3,0.5,0.174957,0.153615,0.129962
4,4,0.5,0.101343,0.094785,0.116129


In [ ]:
en_df.describe()

,index,units_sold_month,xgb_pred_v1,lgb_pred_v1,cat_pred_v1
count,214200.000000,214200.0,214200.000000,214200.000000,214200.000000
mean,107099.500000,0.5,0.293398,0.261340,0.243265
std,61834.358168,0.0,2.157974,2.084328,1.999984
min,0.000000,0.5,-0.481903,-3.439578,-1.181880
25%,53549.750000,0.5,0.036681,0.054728,0.106885
50%,107099.500000,0.5,0.097680,0.094209,0.116235
75%,160649.250000,0.5,0.221586,0.175155,0.153561
max,214199.000000,0.5,390.247742,428.613373,470.916199


In [ ]:
en_df[en_df['xgb_pred_v1']>390]

,index,units_sold_month,xgb_pred_v1,lgb_pred_v1,cat_pred_v1
83339,83339,0.5,390.247742,428.613373,402.958008


In [ ]:
en_df[en_df['cat_pred_v1']>460]

,index,units_sold_month,xgb_pred_v1,lgb_pred_v1,cat_pred_v1
98974,98974,0.5,388.169647,402.239563,470.916199


In [ ]:
print(en_df[en_df['xgb_pred_v1']>= 20].shape)
print(en_df[en_df['lgb_pred_v1']>= 20].shape)
print(en_df[en_df['cat_pred_v1']>= 20].shape)

(81, 5)
(61, 5)
(56, 5)


In [ ]:
print(en_df[en_df['xgb_pred_v1']>= 1].shape)
print(en_df[en_df['lgb_pred_v1']>= 1].shape)
print(en_df[en_df['cat_pred_v1']>= 1].shape)

(10823, 5)
(9437, 5)
(5261, 5)


In [ ]:
print(en_df[en_df['xgb_pred_v1'] < 0].shape)
print(en_df[en_df['lgb_pred_v1'] < 0].shape)
print(en_df[en_df['cat_pred_v1'] < 0].shape)

(9792, 5)
(45, 5)
(2, 5)


In [ ]:
# en_df['xgb_pred_v1'] = en_df['xgb_pred_v1'].apply(lambda x: -9999 if x<0 else x)
# en_df['lgb_pred_v1'] = en_df['lgb_pred_v1'].apply(lambda x: -9999 if x<0 else x)
# en_df['cat_pred_v1'] = en_df['cat_pred_v1'].apply(lambda x: -9999 if x<0 else x)

In [ ]:
print(en_df[en_df['xgb_pred_v1'] == 0].shape)
print(en_df[en_df['lgb_pred_v1'] == 0].shape)
print(en_df[en_df['cat_pred_v1'] == 0].shape)

(0, 5)
(0, 5)
(0, 5)


In [ ]:
submission = pd.read_csv('sample_submission.csv')
# submission['units_sold_month'] = en_df['xgb_pred_v1']*0.80 + en_df['lgb_pred_v1']*0.20
submission['units_sold_month'] = en_df['xgb_pred_v1']*0.80 + en_df['lgb_pred_v1']*0.15 + en_df['cat_pred_v1']*0.05

In [ ]:
submission['units_sold_month'] = submission['units_sold_month'].apply(lambda x: 0 if x<0 else x)

In [ ]:
submission.describe()

,index,units_sold_month
count,214200.000000,214200.000000
mean,107099.500000,0.286138
std,61834.358168,2.127734
min,0.000000,0.000000
25%,53549.750000,0.044351
50%,107099.500000,0.098453
75%,160649.250000,0.213456
max,214199.000000,396.638100


In [ ]:
submission.head()

,index,units_sold_month
0,0,0.157008
1,1,0.019118
2,2,0.296142
3,3,0.169506
4,4,0.101099


In [ ]:
submission[submission['units_sold_month']< 0.01].shape

(6662, 2)

In [ ]:
submission[submission['units_sold_month']> 20].shape

(73, 2)

In [ ]:
submission[submission['units_sold_month']>= 1].shape

(10182, 2)

In [ ]:
submission[submission['units_sold_month'] == 0].shape

(1484, 2)

## download local

In [ ]:
from google.colab import files
submission.to_csv('submission.csv', index=False)
files.download('submission.csv')